In [0]:
!pip install tensorflow==2.1.0rc

In [0]:
!pip install grpcio==1.24.3

In [0]:
!pip install google-auth==1.4.0

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.keras.layers import *
def Model(input_shape,output_shape):
  model = tf.keras.Sequential()
  model.add(Conv2D(4,(2,2),padding = 'same',input_shape = input_shape))
  model.add(MaxPool2D())
  model.add(Conv2D(8,(2,2),padding = 'same',activation='relu'))
  model.add(MaxPool2D())
  model.add(Conv2D(16,(2,2),padding = 'same',activation='relu'))
  model.add(MaxPool2D())
  model.add(Conv2D(32,(2,2),padding = 'same',activation='relu'))
  model.add(Conv2D(64,(2,2),padding = 'same' , activation = 'sigmoid'))
  model.add(Reshape((600,400)))
  return model

In [0]:
entropy = tf.keras.losses.MeanSquaredError()
import cv2
def loss(output,mask):
  return entropy(mask,output)

In [0]:
from zipfile import ZipFile

In [0]:
with ZipFile('/content/sky.zip','r') as zipObj:
  zipObj.extractall()

In [0]:
import os
import numpy as np
import cv2
images = os.listdir('/content/sky/data')
opt = os.listdir('/content/sky/groundtruth')
def Gen(i):
  img = cv2.imread('/content/sky/data/' + images[i])
  img = cv2.resize(img, (600,400), interpolation = cv2.INTER_AREA)
  img = (img / 127.5) - 1.0
  imgr = []
  imgr.append(img)
  img = tf.convert_to_tensor(imgr, dtype=tf.float32)
  opt_img = cv2.imread('/content/sky/groundtruth/' + opt[i])
  opt_img = cv2.resize(opt_img, (600,400), interpolation = cv2.INTER_AREA)
  opt_img = ((opt_img[:,:,0]+opt_img[:,:,1]+opt_img[:,:,2]) / (255.0 *3))
  imgr = []
  imgr.append(opt_img)
  opt_img = tf.convert_to_tensor(imgr, dtype=tf.float32)
  return img , opt_img

In [0]:
EPOCHS = 500
optimizer = tf.keras.optimizers.Adam(1e-4)
import matplotlib.pyplot as plt
def train(model , epochs):
  e_loss = 'INFINITE'
  for e in range(epochs):
    print(e_loss)
    for i in range(len(images)):
      t,o = Gen(i)
      with tf.GradientTape() as tape:
        o_m = model(t)
        lo = loss(o_m , o)
        if i == 0:
          e_loss = lo
        else:
          e_loss += lo
        grads = tape.gradient(lo,model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    if e%10 == 0:
      model.save_weights('/content/')
      z,x = Gen(0)
      plt.imshow(z[0])
      plt.imshow(x[0])
      c = model(z)
      plt.imshow(c[0])

In [8]:
t,o = Gen(0)
Skyfinder = Model(t[0].shape,o[0].shape)
Skyfinder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 600, 400, 4)       52        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 300, 200, 4)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 300, 200, 8)       136       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 100, 8)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 100, 16)      528       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 50, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 50, 32)        2

In [9]:
train(Skyfinder,EPOCHS)

INFINITE
tf.Tensor(9.217433, shape=(), dtype=float32)
tf.Tensor(7.6667423, shape=(), dtype=float32)
tf.Tensor(4.713748, shape=(), dtype=float32)
tf.Tensor(2.4399536, shape=(), dtype=float32)
tf.Tensor(1.8820826, shape=(), dtype=float32)
tf.Tensor(1.7661301, shape=(), dtype=float32)
tf.Tensor(1.7060323, shape=(), dtype=float32)
tf.Tensor(1.659509, shape=(), dtype=float32)
tf.Tensor(1.6202774, shape=(), dtype=float32)
tf.Tensor(1.587308, shape=(), dtype=float32)
tf.Tensor(1.5582894, shape=(), dtype=float32)
tf.Tensor(1.5334021, shape=(), dtype=float32)
tf.Tensor(1.5121429, shape=(), dtype=float32)
tf.Tensor(1.493264, shape=(), dtype=float32)
tf.Tensor(1.4768368, shape=(), dtype=float32)
tf.Tensor(1.462271, shape=(), dtype=float32)
tf.Tensor(1.4488689, shape=(), dtype=float32)
tf.Tensor(1.437411, shape=(), dtype=float32)
tf.Tensor(1.42764, shape=(), dtype=float32)
tf.Tensor(1.4192358, shape=(), dtype=float32)
tf.Tensor(1.411381, shape=(), dtype=float32)
tf.Tensor(1.4039723, shape=(), dtyp